In [1]:
%config Completer.use_jedi = False

import configparser
config = configparser.ConfigParser()
config.read('agg-config.txt')


time_step = int(config.get('Configuration', 'time_step'))
time_step2 = int(config.get('Configuration', 'time_step2'))

window_size = int(config.get('Configuration', 'window_size'))
processed_data_folder = config.get('Configuration', 'processed_data_folder')
raw_data_folder = config.get('Configuration', 'raw_data_folder')

ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

activities=config.get('Configuration', 'learn_activities')
activities = [(item.strip()) for item in activities.split(',')]

cross_days=config.get('Configuration', 'cross_days')
cross_days = [(item.strip()) for item in cross_days.split(',')]

sensors=config.get('Configuration', 'sensors')
sensors = [(item.strip()) for item in sensors.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))

user_color={}
for user in users:
    user_color[user]=config.get('Colors', user)
print(user_color)

{'ed9c': 'green', '0001': 'blue', '3d57': 'red'}


In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(t0,tN,ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(t0,tN,tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


def day_time0(t0,tN,ti):
    return day_time(ti)-day_time(t0)+1

def relT(t0,tN,ti):
    return (int)((ti-t0)/time_step2)

def sizeT(t0,tN):
    return relT(t0,tN,tN)+1


def initTime(ini_date,end_date):
    tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())

    t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
    print (ini_date,"Init date is", t0, "day:",day_time(t0,tN,t0), time2str(t0,tN,t0))
    print (end_date,"End date is", tN, "day:",day_time(t0,tN,tN), time2str(t0,tN,tN))

    day=list(range(day_time(t0,tN,t0),day_time(t0,tN,tN)+1))[0]
    return day,t0,tN
    
initTime(ini_date,end_date)


2024-07-07 02:00:00 Init date is 1720310400 day: 19911 2024-07-07 02:00:00
2024-07-08 01:59:59 End date is 1720396799 day: 19912 2024-07-08 01:59:59


(19911, 1720310400, 1720396799)

In [3]:
from datetime import datetime, timedelta

days=[]
t0N_days={}

for cross_day in cross_days:
    print(cross_day)
    # Convertir las fechas a objetos datetime
    ini_date_dt = datetime.strptime(ini_date, "%Y-%m-%d %H:%M:%S")
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S")
    cross_day_dt = datetime.strptime(cross_day, "%Y-%m-%d")

    # Reemplazar el día manteniendo la hora original
    new_ini_date = cross_day_dt.replace(hour=ini_date_dt.hour, minute=ini_date_dt.minute, second=ini_date_dt.second)
    new_end_date = cross_day_dt.replace(hour=end_date_dt.hour, minute=end_date_dt.minute, second=end_date_dt.second) + timedelta(days=1)

    # Convertir las fechas de nuevo a cadena si es necesario
    new_ini_date_str = new_ini_date.strftime("%Y-%m-%d %H:%M:%S")
    new_end_date_str = new_end_date.strftime("%Y-%m-%d %H:%M:%S")

    print(f"New ini_date: {new_ini_date_str}")
    print(f"New end_date: {new_end_date_str}")
    day,t0,tN=initTime(new_ini_date_str,new_end_date_str)
    days.append(day)
    t0N_days[day]=(t0,tN)
    

print(days)
print(t0N_days)



2024-07-05
New ini_date: 2024-07-05 02:00:00
New end_date: 2024-07-06 01:59:59
2024-07-05 02:00:00 Init date is 1720137600 day: 19909 2024-07-05 02:00:00
2024-07-06 01:59:59 End date is 1720223999 day: 19910 2024-07-06 01:59:59
2024-07-06
New ini_date: 2024-07-06 02:00:00
New end_date: 2024-07-07 01:59:59
2024-07-06 02:00:00 Init date is 1720224000 day: 19910 2024-07-06 02:00:00
2024-07-07 01:59:59 End date is 1720310399 day: 19911 2024-07-07 01:59:59
2024-07-07
New ini_date: 2024-07-07 02:00:00
New end_date: 2024-07-08 01:59:59
2024-07-07 02:00:00 Init date is 1720310400 day: 19911 2024-07-07 02:00:00
2024-07-08 01:59:59 End date is 1720396799 day: 19912 2024-07-08 01:59:59
[19909, 19910, 19911]
{19909: (1720137600, 1720223999), 19910: (1720224000, 1720310399), 19911: (1720310400, 1720396799)}


In [4]:
import os
import pandas as pd


for ax,activity in enumerate(activities):
    print("\t",activity)
    for ux,user in enumerate(users):
        for dx,day in enumerate(days):
            series=read_csv(processed_data_folder+"/DAY_"+str(day)+"/act/loc."+user+"."+activity+".tsv", sep="\t",parse_dates=True, header=None, names=["time","loc"])

    


	 resting
	 exit
	 cooking
	 toileting
	 shower
	 pc
	 sleep
	 kitchen


In [5]:
act_sensors=np.zeros((len(sensors),len(days),sizeT(t0,tN)))
print(act_sensors.shape)
for sx,sensor in enumerate(sensors):
    print("\t",sensor)
    for dx,day in enumerate(days):
        print(processed_data_folder+"/DAY_"+str(day)+"/object_sensor/"+sensor+".tsv")
        series=read_csv(processed_data_folder+"/DAY_"+str(day)+"/object_sensor/"+sensor+".tsv", sep="\t",parse_dates=True, header=None, names=["time","status"])
        series=series["status"].values
        print(day,series.shape)
        act_sensors[sx][dx]=series

(11, 3, 1440)
	 fridge_13
./data-har//DAY_19909/object_sensor/fridge_13.tsv
19909 (1440,)
./data-har//DAY_19910/object_sensor/fridge_13.tsv
19910 (1440,)
./data-har//DAY_19911/object_sensor/fridge_13.tsv
19911 (1440,)
	 tap_22
./data-har//DAY_19909/object_sensor/tap_22.tsv
19909 (1440,)
./data-har//DAY_19910/object_sensor/tap_22.tsv
19910 (1440,)
./data-har//DAY_19911/object_sensor/tap_22.tsv
19911 (1440,)
	 micro_5
./data-har//DAY_19909/object_sensor/micro_5.tsv
19909 (1440,)
./data-har//DAY_19910/object_sensor/micro_5.tsv
19910 (1440,)
./data-har//DAY_19911/object_sensor/micro_5.tsv
19911 (1440,)
	 pans_8
./data-har//DAY_19909/object_sensor/pans_8.tsv
19909 (1440,)
./data-har//DAY_19910/object_sensor/pans_8.tsv
19910 (1440,)
./data-har//DAY_19911/object_sensor/pans_8.tsv
19911 (1440,)
	 dishes_9
./data-har//DAY_19909/object_sensor/dishes_9.tsv
19909 (1440,)
./data-har//DAY_19910/object_sensor/dishes_9.tsv
19910 (1440,)
./data-har//DAY_19911/object_sensor/dishes_9.tsv
19911 (1440,)
	 

In [ ]:
user_activity=np.zeros((len(activities),len(users),len(days),sizeT(t0,tN)))
for ax,activity in enumerate(activities):
    for ux,user in enumerate(users):
        for dx,day in enumerate(days):
            print(processed_data_folder+"/DAY_"+str(day)+"/act/INTERVAL."+user+"."+activity+".tsv")
            df_total=read_csv(processed_data_folder+"/DAY_"+str(day)+"/act/INTERVAL."+user+"."+activity+".tsv", sep="\t",parse_dates=True, header=None, names=["ix","d0","dN"])
            #print(df_total)
            #print(pd.to_datetime(df_total['d0']))
            df_total['d0'] = pd.to_datetime(df_total['d0'])
            df_total['dN'] = pd.to_datetime(df_total['dN'])
            #print(df_total)

            (t0,tN)=t0N_days[day]
            ts=list(range(t0,tN,time_step2))
            
            for tx,tt in enumerate(ts):
                ttt=pd.to_datetime(time2str(t0,tN,tt))
#                print(ttt)
                row = df_total[(df_total['d0'] <= ttt) & (df_total['dN'] >= ttt)]
#                print("df_in#tervalo",row)
                if(row.size>0):
                    user_activity[ax][ux][dx][tx]=1

for ax,activity in enumerate(activities):
    for ux,user in enumerate(users):
        for dx,day in enumerate(days):
            (t0,tN)=t0N_days[day]
            ts=list(range(t0,tN,time_step2))
            with open(processed_data_folder+"/DAY_"+str(day)+"/act/LABEL3."+user+"."+activity+".tsv", "w") as file:
                for tx,tt in enumerate(ts):
                    file.write(time2str(t0,tN,tt)+"\t"+str(user_activity[ax][ux][dx][tx])+"\n")




./data-har//DAY_19909/act/INTERVAL.ed9c.resting.tsv
./data-har//DAY_19910/act/INTERVAL.ed9c.resting.tsv
./data-har//DAY_19911/act/INTERVAL.ed9c.resting.tsv
./data-har//DAY_19909/act/INTERVAL.0001.resting.tsv
./data-har//DAY_19910/act/INTERVAL.0001.resting.tsv
./data-har//DAY_19911/act/INTERVAL.0001.resting.tsv
./data-har//DAY_19909/act/INTERVAL.3d57.resting.tsv
./data-har//DAY_19910/act/INTERVAL.3d57.resting.tsv
./data-har//DAY_19911/act/INTERVAL.3d57.resting.tsv
./data-har//DAY_19909/act/INTERVAL.ed9c.exit.tsv
./data-har//DAY_19910/act/INTERVAL.ed9c.exit.tsv
./data-har//DAY_19911/act/INTERVAL.ed9c.exit.tsv
./data-har//DAY_19909/act/INTERVAL.0001.exit.tsv
./data-har//DAY_19910/act/INTERVAL.0001.exit.tsv
./data-har//DAY_19911/act/INTERVAL.0001.exit.tsv
./data-har//DAY_19909/act/INTERVAL.3d57.exit.tsv
./data-har//DAY_19910/act/INTERVAL.3d57.exit.tsv
./data-har//DAY_19911/act/INTERVAL.3d57.exit.tsv
./data-har//DAY_19909/act/INTERVAL.ed9c.cooking.tsv
./data-har//DAY_19910/act/INTERVAL.ed9c

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Flatten, concatenate, Reshape
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Lambda, Dot,Reshape,Multiply, Input, Conv1D, Conv2D, ConvLSTM2D,ReLU,UpSampling2D, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D, Concatenate


N1=len(activities)
N2=len(activities)
N3=len(sensors)

TWA=30
TWB=15
T=TWA+TWB

def getModelHAR():
    # Definición de las entradas
    input1 = Input(shape=(N1, T, 1))
    input2 = Input(shape=(N2, T, 1))
    input3 = Input(shape=(N3, T, 1))

    # Bloque para la primera secuencia
    conv1_1 = TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))(input1)
    conv1_2 = TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))(conv1_1)
    flat1 = TimeDistributed(Flatten())(conv1_2)

    # Bloque para la segunda secuencia
    conv2_1 = TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))(input2)
    conv2_2 = TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))(conv2_1)
    flat2 = TimeDistributed(Flatten())(conv2_2)

    # Bloque para la tercera secuencia
    conv3_1 = TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))(input3)
    conv3_2 = TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))(conv3_1)
    flat3 = TimeDistributed(Flatten())(conv3_2)

    # Concatenación de los tres bloques
    merged = concatenate([flat1, flat2, flat3],axis=1)

    # Primera capa LSTM después de la concatenación
    lstm_merged1 = LSTM(256, return_sequences=True)(merged)

    # Segunda capa LSTM después de la concatenación
    lstm_merged2 = LSTM(512, return_sequences=False)(lstm_merged1)

    # Capa densa para la clasificación binaria
    dense = Dense(128, activation='relu')(lstm_merged2)
    output = Dense(1, activation='sigmoid')(dense)

    # Definición del modelo
    model = Model(inputs=[input1, input2, input3], outputs=output)

    # Compilación del modelo
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Instanciación del modelo
model = getModelHAR()
model.summary()

In [ ]:
import random



SizeTrainOk=1000
SizeTrainKo=1000


for ax,activity in enumerate(activities):
        print(ax,activity)
        for dx,day in enumerate(days):
            print("\t",dx,day)
            other_dx= [other for other in range(0,len(days)) if other != dx]
            
            ##TRAING DATA
            L1=[]
            L2=[]
            S=[]
            Y=[]
            for ux,user in enumerate(users):
                    print("\t\t",ux,user)
                    other_ux= [other for other in range(0,len(users)) if other != ux]
                    print(day,user,other_dx,other_ux)

                    #OK act
                    for ix in range(0,SizeTrainOk):
                        #print("SizeTrainOk",ix,"vs",SizeTrainOk)
                        filtered_elements = user_activity[ax, ux, other_dx, :]

                        # Encontramos los elementos que son 1
                        indices = np.argwhere(filtered_elements == 1)

                        if(len(indices)==0):
                            #print("\t\t\t","Non activity for user ")
                            break
                        
                        # Para obtener las posiciones relativas a las dimensiones originales
                        indexs = [(ax, ux, other_dx[idx[0]], idx[1]) for idx in indices]
                        index = random.choice(indexs)
                        #print("\t\t\t",index,user_activity[index])
                        
                        (ia,iu,idy,tt)=index
                        
                        if(tt<TWA):
                            tt=TWA
                        if(tt>=sizeT(t0,tN)-TWB):
                            tt=sizeT(t0,tN)-TWB
                        
                        
                        X1s=[]
                        X2s=[]
                        for ax2,activity2 in enumerate(activities):
                            X1s.append(loc_user[ax2][iu][idy][tt-TWA:tt+TWB])
                            X2s.append(np.amax(loc_user[ax2,other_ux,idy,tt-TWA:tt+TWB],axis=0))
                            
                            
                        Ss=[]
                        for sx, sensor in enumerate(sensors):
                            Ss.append(act_sensors[sx][idy][tt-TWA:tt+TWB])
                        L1.append(X1s)
                        L2.append(X2s)
                        S.append(Ss)
                        Y.append(user_activity[index])
                        
                    #KO act
                    ix=0
                    while ix < SizeTrainKo:
                        #print("SizeTrainKo",ix,"vs",SizeTrainKo)
                        filtered_elements_specific = user_activity[ax, ux, other_dx, :]

                        # Encontramos los elementos que son 1
                        indices = np.argwhere(filtered_elements == 0)

                        if(len(indices)==0):
                            print("\t\t\t","Non activity for user ")
                            break
                        
                        # Para obtener las posiciones relativas a las dimensiones originales
                        indexs = [(ax, ux, other_dx[idx[0]], idx[1]) for idx in indices]
                        index = random.choice(indexs)
                        #print("\t\t\t",index,user_activity[index])
                        
                        (ia,iu,idy,tt)=index
                        
                        if(tt<TWA):
                            tt=TWA
                        if(tt>=sizeT(t0,tN)-TWB):
                            tt=sizeT(t0,tN)-TWB
                        
                        
                        X1s=[]
                        X2s=[]
                        none=False
                        for ax2,activity2 in enumerate(activities):
                            if(ax2==ax):
                                r=np.random.rand()
                                #print(" removing loc?:",loc_user[ax2][iu][idy][tt],r)
                                if(r>loc_user[ax2][iu][idy][tt]+0.05):
                                    #print("YES  removing loc:",loc_user[ax2][iu][idy][tt])
                                    none=True
                                    break
                            X1s.append(loc_user[ax2][iu][idy][tt-TWA:tt+TWB])
                            X2s.append(np.amax(loc_user[ax2,other_ux,idy,tt-TWA:tt+TWB],axis=0))
                            
                            
                        Ss=[]
                        for sx, sensor in enumerate(sensors):
                            Ss.append(act_sensors[sx][idy][tt-TWA:tt+TWB])
                        if not none:
                            L1.append(X1s)
                            L2.append(X2s)
                            S.append(Ss)
                            Y.append(user_activity[index])       
                            ix=ix+1

            L1=np.array(L1)        
            L2=np.array(L2)        
            S=np.array(S)
            Y=np.array(Y)
            print("TRAIN",activity,day,L1.shape,L2.shape,S.shape,Y.shape)
            
            
            model=getModelHAR()
            model.summary()
            
            ##TEST DATA
            TL1=[]
            TL2=[]
            TS=[]
            TY=[]
            
            
            (t0,tN)=t0N_days[day]
            ts=list(range(TWA,sizeT(t0,tN)-TWB))
            
            for ux2,user2 in enumerate(users):
                other_ux2= [other for other in range(0,len(users)) if other != ux2]

                for tx in ts:

                    Ss=[]
                    for sx, sensor in enumerate(sensors):
                        Ss.append(act_sensors[sx][dx][tx-TWA:tx+TWB])
                    
                    X1s=[]
                    X2s=[]
                    for ax2,activity2 in enumerate(activities):
                        X1s.append(loc_user[ax2][ux2][dx][tx-TWA:tx+TWB])
                        X2s.append(np.amax(loc_user[ax2,other_ux2,dx,tx-TWA:tx+TWB],axis=0))


                    TL1.append(X1s)
                    TL2.append(X2s)
                    TS.append(Ss)
                    TY.append(user_activity[ax, ux2, dx, tx])
                        
            TL1=np.array(TL1)        
            TL2=np.array(TL2)        
            TS=np.array(TS)
            TY=np.array(TY)
            print("EVAL",activity,day,TL1.shape,TL2.shape,TS.shape,TY.shape)
            
            history = model.fit([L1,L2,S] , Y, shuffle=True, batch_size=32,epochs=50, validation_data = ([TL1,TL2,TS], TY))
            YT_prediction = model.predict([TL1,TL2,TS])
            print(YT_prediction.shape)
            
            ix=0
            for ux2,user2 in enumerate(users):
                with open(processed_data_folder+"/DAY_"+str(day)+"/act/PREDICTION."+user2+"."+activity+".tsv", "w") as file:
                    for tx in (ts):
                        tt=tx*time_step2+t0
                        #print("\t"+time2str(t0,tN,tt)+"\t"+str(TY[ix])+"\t"+str(YT_prediction[ix][0]))
                        file.write("\t"+time2str(t0,tN,tt)+"\t"+str(TY[ix])+"\t"+str(YT_prediction[ix][0])+"\n")
                        
                        ix=ix+1


